In [1]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

In [2]:
# Create a temporary directory to store the code files.
temp_dir = tempfile.TemporaryDirectory()

# Create a local command line code executor.
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

In [3]:
# Create an agent with code executor configuration.
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
)

In [4]:
message_with_code_block = """This is a message with code block.
The code block is below:
```python
import formulallm.formula as f
dir(f)
f.help()
code = f.load("./data/MappingExample.4ml")
f.solve("pm","1","Mapping.conforms")
f.list()
f.extract("0","0","0")
```
This is the end of the message.
"""

In [5]:
reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)

Provide feedback to the sender. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: apply (ap)             - Start an apply task. Use: apply transformstep
confhelp (ch)          - Provides help about module configurations and settings
core (cr)              - Prints reduced rule set for domains / transforms. Use: core module_name
del (d)                - Deletes a variable. Use: del var.
det (dt)               - Prints details about the compiled module with the given name. Use: det modname
downgrade (dg)         - Attempts to downgrade a (partial) model to Formula V1. Use: downgrade module_name
exit (x)               - Exits the interface loop.
extract (ex)           - Extract and install a result. Use: extract (app_id | solv_id n) output_name [render_class render_dll]
generate (gn)          - Generate C# data model. Use: generate modname
help (h)               - Prints this message.
interactive